In [1]:
struct Economy_Dynare{T<:Real,I<:Int64}
    
    beta::T
    alpha::T
    delta::T
    gamma::T
    chi::T
    phi::T
    taul::T
    tauk::T
    taus::T
    Tt::T
    
    psip::T
    psiw::T
    epsp::T
    epsw::T
    kappaw::T
    
    na    ::I # number of grid points
    a_min ::T         # min asset holdings
    aGrid ::Vector{T} # grid for asset choices

    #=
        *** productivity process ***
    =#
    ny    ::I         #number of states
    ys    ::Vector{T} #productivity states    (States)
    Piy   ::Matrix{T} #Transition matrix (under vectorial format) (Transv)
    Sy    ::Vector{T} #distribution ver y size is ns (disty)
    #yvect ::Array{T,1} #vector of y size is ns*na
end
function Economy_Dynare(economy::Economy)
    @unpack β,α,δ,γ,χ,ϕ,τs,τl,τk,Tt,ψp,ψw,ϵp,ϵw,κw,
        na,a_min,aGrid,
        ny,ys,Πy,Sy = economy
    Economy_Dynare(β,α,δ,γ,χ,ϕ,τl,τk,τs,Tt,ψp,ψw,ϵp,ϵw,κw,na,a_min,aGrid,ny,ys,Πy,Sy)
end

LoadError: LoadError: UndefVarError: `@unpack` not defined
in expression starting at /Users/am13232/Dropbox/optimal_money_New/Code_Wage/AMB/Sticky_Wages/steady-state/ToDynare_SW.ipynb:34

In [2]:
struct TruncatedAllocation_Dynare{I<:Integer,T<:Real}
    S_h::Vector{T}       # size of truncated histories
    Pi_h::Matrix{T}    # transition matrix for histories
    y0_h::Vector{T}      # current productivity levels
    a_beg_h::Vector{T}   # beginning-of-period wealth per history and per capita
    a_end_h::Vector{T}   # end-of-period wealth per history and per capita
    c_h::Vector{T}       # consumption per history and per capita
    l_h::Vector{T}       # labor supply per history and per capita
    ly_tau_h::Vector{T}    # (l y)^τ
    u_h::Vector{T}       # utility of consumption per history and per capita
    uc_h::Vector{T}      # marginal utility of consumption per history and per capita
    ucc_h::Vector{T}     # second-order derivative of utility of consumption per history and per capita
    v_h::Vector{T}       # utility of labor per history and per capita
    vl_h::Vector{T}      # marginal utility of labor per history and per capita
    resid_E_h::Vector{T} # Euler Lagrange multiplier at the history level
    nb_cc_h::I           # nb of credit constrained histories
    ind_cc_h::Vector{I}  # indices of credit constrained histories
end

struct xsis_struct_Dynare{T<:Real}
    xsiu0::Vector{T}
    xsiu1::Vector{T}
    xsiu2::Vector{T}
    xsiuE::Vector{T}
    xsiy ::Vector{T}
    xsiv0::Vector{T}
    xsiv1::Vector{T}
end  

struct TruncatedModel_Dynare{I<:Integer,T<:Real}
    N::I
    refiNs::Vector{I}
    Ntot::I
    ind_h::Vector{I}
    truncatedAllocation::TruncatedAllocation_Dynare{I,T}
    xsis::xsis_struct_Dynare{T}
end
function TruncatedAllocation_Dynare(truncatedAllocation::TruncatedAllocation)
    @unpack S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,ly_τ_h,u_h,u′_h,u′′_h,
            v_h,v′_h,resid_E_h, nb_cc_h,ind_cc_h = truncatedAllocation
    TruncatedAllocation_Dynare(S_h,Matrix(Π_h),y0_h,a_beg_h,a_end_h,c_h,l_h,ly_τ_h,u_h,u′_h,u′′_h,
            v_h,v′_h,resid_E_h, nb_cc_h,ind_cc_h)    
end
function xsis_struct_Dynare(ξs::ξs_struct)
    @unpack ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1 = ξs
    xsis_struct_Dynare(ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1)
end
function TruncatedModel_Dynare(truncatedModel::TruncatedModel)
    @unpack N,refiNs,Ntot,ind_h,truncatedAllocation,ξs = truncatedModel
    TruncatedModel_Dynare(N,refiNs,Ntot,ind_h,
        TruncatedAllocation_Dynare(truncatedAllocation),xsis_struct_Dynare(ξs))
end

In [3]:
struct Weights_Dynare{T<:Real}
    omega       ::Vector{T} 
    omegab      ::Vector{T}
    omega_h     ::Vector{T}
    omegab_h    ::Vector{T}
    omega_param ::Vector{T} 
    omegab_param::Vector{T}
end 
struct LagrangeMult_Dynare{T<:Real}
    lambdac ::Vector{T}
    lambdact::Vector{T}
    lambdacb::Vector{T}
    # lambdal ::Vector{T}
    # lambdalt::Vector{T}
    # lambdalb::Vector{T}
    mu      ::T
    Lambda  ::T
    gammaw  ::T
    upsilon ::T
    Gamma   ::T
    psi     ::Vector{T}
    psib    ::Vector{T}
end 
struct Ramsey_Dynare{I<:Integer,T<:Real}
    # weights       ::Weights_Dynare{T}
    lagrangeMult  ::LagrangeMult_Dynare{T}
    truncatedModel::TruncatedModel_Dynare{I,T}
end
function Weights_Dynare(weights::Weights)
    @unpack ω,ωb,ω_h,ωb_h,ω_param,ωb_param = weights
    Weights_Dynare(ω,ωb,ω_h,ωb_h,ω_param,ωb_param)
end
function LagrangeMult_Dynare(lagrangeMult::LagrangeMult)
    @unpack λc,λct,λcb,μ,Λ,γw,υ,Γ,ψ,ψb = lagrangeMult
    LagrangeMult_Dynare(λc,λct,λcb,μ,Λ,γw,υ,Γ,ψ,ψb)
end
function Ramsey_Dynare(ramsey::Ramsey)
    @unpack lagrangeMult,truncatedModel = ramsey
    Ramsey_Dynare(LagrangeMult_Dynare(lagrangeMult),
                  TruncatedModel_Dynare(truncatedModel))
end

In [4]:
function Write_Dynare(filename::String,ramsey::Ramsey,solution::AiyagariSolution,economy::Economy)::Nothing
    
    filename_elts = split(filename,".")
    if (length(filename_elts) == 1) 
        filename_ = filename*".mat"
    elseif (length(filename_elts) > 2) 
        filename_ = join(filename_elts[1:end-1],"-")*".mat"
    elseif (filename[end]) != "mat"
        filename_ = join(filename_elts[1:end-1],"-")*".mat"
    else
        filename_ = filename
    end
        
    economy_Dynare = Economy_Dynare(economy)
    ramsey_Dynare  = Ramsey_Dynare(ramsey)
    
#     @unpack weights,lagrangeMult,truncatedModel = ramsey
#     @unpack N,Ntot,ind_h,truncatedAllocation,ξs = truncatedModel
#     @unpack S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,ly_τ_h,u_h,u′_h,u′′_h,v_h,v′_h,nb_cc_h,ind_cc_h = (
#                 truncatedAllocation)
#     @unpack ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1 = ξs 
#     @unpack β,α,δ,τ,u′,u′′,v′,v′′,ys,ny = economy
#     @unpack w,R,K,L,G,Y,A = solution
#     @unpack λcb,ψb,λlb,λc,λct,λl,λlt,ψ,μ = lagrangeMult
#     @unpack ωb_h,ω_h = weights

    file = matopen(filename, "w")
    write(file, "Ramsey", ramsey_Dynare) 
    write(file, "eco", economy_Dynare)
    write(file, "solution", solution)
    close(file)
end

In [ ]:

#=
# Res = proj.allocation_proj.u′_h - (params.β*solution.R)*(proj.allocation_proj.Π_h)*proj.allocation_proj.u′_h
#  and xsie ? 


tauopt = -params.Tt/(solution.K^params.α*solution.L^(1-params.α))
Ctot = w*L -A + Tt + R*sum(solution.stationaryDist.*repeat(params.aGrid,1,ny)) # add L here AMB
# Ws = sum(proj.ξs.ξu1 .*proj.allocation_proj.S_h.*proj.allocation_proj.u_h) .+(-Tt)^(params.θ)


# AMB : Attention ici c'est pas propre, à corriger ,  
#Res = test(3,solution,params);

Res = proj.allocation_proj.resid_E_h


eco = Projection_dd(
    proj.N,
    params.ny,
    proj.Ntot,
    solution.R,
    solution.w,
    params.Tt, # pb réglé
    solution.A,
    solution.B,
    params.ys,
    params.tk,
    params.tl,
    collect(vec(params.Πy')),
    proj.allocation_proj.S_h,
    proj.allocation_proj.S_h .*proj.allocation_proj.a_beg_h,#by bin
    proj.allocation_proj.S_h .*proj.allocation_proj.a_end_h,#by bin
    ones(5),
    proj.allocation_proj.S_h .*proj.allocation_proj.c_h,#by bin
    proj.allocation_proj.S_h .*proj.allocation_proj.l_h,# l by bin : I add it here
    proj.allocation_proj.u′_h ./proj.ξs.ξu1, #vector of marginal utilities (before multiplication by xsip) by bin
    Matrix(proj.allocation_proj.Π_h'), # transpose transition matrix !!!!
    proj.allocation_proj.u′_h .*proj.allocation_proj.S_h,
    params.u.(proj.allocation_proj.c_h) .+10,
    Res,  # attention celui-la est faux! vérifier d'ou vient l'erreur
    proj.ind_h,
    proj.allocation_proj.resid_E_h, # celui-la est bon, c'est Resh dans proj_dd et donc dans dynare ; change proj_d here, pb with res AMB
    proj.allocation_proj.ind_cc_h*0,
    proj.allocation_proj.ind_cc_h,
    proj.allocation_proj.y0_h,
    proj.ξs.ξuE,
    proj.ξs.ξy,
    proj.ξs.ξu0,
    proj.ξs.ξu1,
    proj.ξs.ξu2,
    proj.ξs.ξv0,
    proj.ξs.ξv1
    )


# computing the pricing kernel
# M = sum(eco.Ucv.*eco.xsip.*Pl.omega.*eco.Sp)
# à vérifier

M = sum(eco.Ucv.*eco.xsiue.*eco.Sp)

varPSI = sum(Sy.*(ys.^ν))
    
# file = matopen("/Users/alais/Desktop/todynare_Truncation.mat", "w")
file = matopen("todynare_Truncation_revision.mat", "w")
write(file, "Plann", plan) 
write(file, "eco", eco)
write(file, "alpha", params.α)
write(file, "nu", params.ν)
write(file, "varPSI", varPSI)
write(file, "Ltot", solution.L)
write(file, "kappa", params.κ)
write(file, "beta", params.β) 
write(file, "phi", params.ϕ)  
write(file, "chi", params.χ) 
write(file, "delta", params.δ)
write(file, "gamma", params.γ) 
write(file, "epss", params.ϵ)
write(file, "tk", params.tk)
write(file, "tl",params.tl)
write(file, "abar", params.a_min)
write(file, "states", params.ys)
# write(file, "G", -params.Tt)
write(file, "G", solution.G)
write(file, "Ctot", Ctot)
write(file, "M", M)
# write(file, "Ws", Ws)
close(file)

end=#